# LoRA implementation

# Qwen2-VL-7B-Instruct
Qwen/Qwen2-VL-7B-Instruct

In [ ]:
# ! pip install bitsandbytes peft trl
# ! pip install trl

# LoRA: low rank adaption of LLMs

## why we need LoRA?
In nn, we have 
        
        input -> hidden layer1 -> hidden layer2 -> output
        then we campare output and target for loss and then backpropagation to each of the weights of all the layers and each of these weights will modified by the loss function

![alt-txt](finetuning_01.png)
In finetuning we have 
        
        a pretrained model and we want to finetune it on some other data that original model might not have seen. 

problem with fine tuning,

        **Computational Cost**

            Requires updating all model parameters
            High GPU/TPU requirements
            Expensive for large models


        **Memory Issues**

            Full model weights must fit in GPU memory
            Large batch sizes often impossible


        **Catastrophic Forgetting**

            Model may lose previously learned knowledge
            Can degrade performance on original tasks


## LoRA
In LoRA, we start wtht input than we have our pretrained model and we freeze it weight, basically we tell pytorch to never touch these weights; just use them as read-only, never run back propagation on them.
Then we create two other matrices A,B(basically in LoRA we dont have to create matrices A and B for each layer, wecan just do it for some layers).
Suppose we only have one layer, we introduced the matrices A and B

so what's the differece betwenn A and B and original(pre trained) matrix W?

    1. dimension-wise: W is dxk i.e. 1000, 5000; we want to create two new matrices A and B so that multiplied together they produce same dimension `dxk`
        W: dxk => 1000*5000 -> 5 000000 parameters
        
        B: dxr
        A: rxk
        B.A: dxk   where r is much smaller than d and k so supose r =1, parameters = 1000+ 5000 =>  6000


        this matrix may not capture the same infomration as original matrix W as its much smaller even though they produce same dimensions, this is smaller representation. so you loose some infomation.



    Idea behnd LoRA is that : matrix W contains a lot of weights that are meaningful for our purposes, they are actually not adding any information to the model; they are just combination of the other weights so they are redundant. So, we dont need whole matrix W . we can create a lower representation of this W and finetune that one.

![alt-txt](finetuning_02.png)(





**we create two matrices B and A then we combine them with W, because we can sum them because both have same dimensions shape(BA) = shape(W), then we produce the output.**
then we have our usual target(ground truth/label) to compute loss against output. then we only back propagate the loss along the matrix that we want to train B and A. so we never touched the W matrix(original model

![alt-txt](finetuning_03.png))

















### benfits
    1. less parameter to train
        if d=1000, k =5000 => d*k = 5000000 
        using r=5:
        we get (dxr) + (rxk) => 5000 + 25000 = 30000.
    2. less parameters = less storage requirements
    3. fast backpropagation, we do not need to evaluate the graident of the most oof parameters.
    4. while woriking with two differnet fine tuned model(one for java , one for sql) just by chaingingthe parameters of A and B matrices instead of reloading W again becasue W was never touched.








# LoRA
Even though LLMs are enormous, the changes you need to make to them to adapt them for a specific task are surprisingly structures.
Rather than modifying all weights, LoRA inserts small trainsble parameters into each layer of the model. like Instead of rebuilding entire engne of race car, LoRA adds carefully designed tweeks.
It not only reduces model checkpoint size from 350gb to 35 mb but also enhances perfommance.



# the ideal tweak to the model, often has  a very low intrinsic rank. here rank is the measure of complexity of adjustment. meaning very low rank meaning very low adjustment can lead to significant improvemnt. Igt's not about making small tweaks but smart target tweaks.


# if these simple adujustments can unock so much potential, it means their is adeeper structure to how these models learn and represent knowledge.

# its like having really large library, but nnot knowing how to find the right books. LoRA givrs you perfect index guiding you to exact book.

# researches found out that applying LORA to both query(questions) and value(answers) matrices in self attention yielded the best perofmmances.adapting just one or other wasn't jus t as effective.

# rather than adding new info, LORA just amplifies alrady existig parameters i.e. boost in features. so LoRA is just liken a splotlight that highlights the specific knowledge needed for that task at hand.


![alt-txt](finetuning_04.png)


pretrained models have a low intrinsic dimesnion and can still learn efficiently despite a random projection to a smaller subspace. Inspired by this, hypotheseis is the updates of weights also have low intrinsic rank during adaption.

### rank of maatrix is number of vectors that are linearly independent in that matrix i..e you can not combine any of them kinearly to produce an another one. It also indicates how many cikumns are redundant because all redunadnat columns can be obtained by linearly combining others.

## So, W matrix is actually rank deficient whcih mean it doesn't have full rank. i.e. dimenions maybe 1000*1000 but actual rankg is 10, so we can use capture most of information by using 10x10 matrix.



















This image provides insights into the concept of **low-rank parameterized update matrices** in fine-tuning pre-trained language models. Here's a breakdown of the highlighted ideas:

1. **Intrinsic Dimension in Pre-trained Models**:  
   The work by Aghajanyan et al. (2020) demonstrates that pre-trained language models possess a low "intrinsic dimension." This means they can adapt to tasks efficiently even when projected into a smaller subspace, suggesting that their underlying structure doesn't require the full parameter space for effective learning.

2. **Hypothesis on Intrinsic Rank in Adaptation**:  
   Inspired by the above, the authors hypothesize that the **updates to the weight matrices** during task adaptation also have a **low "intrinsic rank."** This aligns with the idea that a smaller rank suffices to capture the essential changes needed for adaptation.

3. **Mathematical Framework**:  
   - The weight matrix \( W_0 \in \mathbb{R}^{d \times k} \) of a pre-trained model is constrained during adaptation by decomposing updates as \( \Delta W = B A \), where \( B \in \mathbb{R}^{d \times r} \) and \( A \in \mathbb{R}^{r \times k} \).  
   - The rank \( r \) is much smaller than \( \min(d, k) \), meaning the updates are **rank-deficient**.  
   - \( W_0 \) remains frozen during training, and only \( A \) and \( B \) are trainable.

4. **Implication**:  
   By adopting this low-rank update approach, models retain efficiency and performance while drastically reducing the number of parameters being fine-tuned. This aligns with efficient fine-tuning methods like **LoRA (Low-Rank Adaptation)**.

Would you like a deeper explanation of any specific part, such as its applications or derivation?




### **In-depth Explanation of LoRA (Low-Rank Adaptation) with Mathematical Foundations**

LoRA (Low-Rank Adaptation) is an efficient fine-tuning method for large language models (LLMs). Instead of updating all the parameters of the model, LoRA introduces a small set of additional parameters that are fine-tuned to adapt the model to a specific task. This reduces computational requirements and storage while maintaining high performance. Let’s explore it step by step with the necessary mathematical background.

---

### **1. Key Idea Behind LoRA**

LLMs are typically parameterized by extremely large matrices \(W\) in each layer. These matrices define the weights that transform inputs to outputs across layers. Fine-tuning traditionally involves updating all these parameters for a specific task, which is computationally expensive and storage-heavy.

LoRA’s key observation: **The task-specific updates to these large matrices have low intrinsic rank.** This means that instead of modifying the full \(W\), the changes can be expressed as a low-rank decomposition.

---

### **2. Mathematical Formulation of LoRA**

Let \(W\) be a weight matrix in the model with dimensions \((d_{\text{out}}, d_{\text{in}})\). During fine-tuning, instead of directly updating \(W\), LoRA expresses the update as:

\[ W' = W + \Delta W \]

Where:
- \(W\) is the pre-trained weight matrix (frozen during LoRA fine-tuning).
- \(\Delta W\) is the task-specific update matrix, which LoRA approximates using a low-rank decomposition.

#### **Low-Rank Decomposition**
The update \(\Delta W\) is parameterized as:

\[ \Delta W = AB^\top \]

Where:
- \(A\) is a matrix of dimensions \((d_{\text{out}}, r)\), where \(r\) is the rank of the decomposition.
- \(B\) is a matrix of dimensions \((d_{\text{in}}, r)\).
- \(r\) is much smaller than the dimensions of \(W\), making \(\Delta W\) a low-rank approximation.

#### **Effective Parameter Reduction**
The number of parameters introduced by LoRA is:

\[ \text{Parameters in } \Delta W = r \cdot (d_{\text{out}} + d_{\text{in}}) \]

This is significantly smaller than the full \(d_{\text{out}} \cdot d_{\text{in}}\) parameters of \(W\).

---

### **3. Training with LoRA**

During fine-tuning:
- \(W\) is frozen, meaning it is not updated.
- Only the parameters \(A\) and \(B\) are trained, which significantly reduces computational costs.

The forward pass with LoRA becomes:

\[ y = Wx + (AB^\top)x \]

Where:
- \(x\) is the input vector of size \(d_{\text{in}}\).
- \(y\) is the output vector of size \(d_{\text{out}}\).

#### **Backpropagation**
Only the gradients for \(A\) and \(B\) are computed during backpropagation, while \(W\) remains unchanged. This further reduces the memory and computational overhead.

---

### **4. Intrinsic Rank and Task-Specific Adaptation**

#### **Intrinsic Rank Hypothesis**
LoRA is based on the hypothesis that task-specific updates to a pre-trained LLM are inherently low-rank. This is because:
- Pre-trained models already encode a vast amount of general knowledge.
- Fine-tuning for a specific task typically requires only minor adjustments to highlight the relevant features.

The rank \(r\) determines the expressiveness of the update. Empirical studies have shown that even very small values of \(r\) (e.g., \(r = 4\) or \(r = 8\)) can lead to significant performance improvements.

---

### **5. Application in Self-Attention Mechanism**

LoRA has been particularly effective when applied to the **query** and **value** matrices in the self-attention mechanism of transformers.

#### **Attention Mechanism Overview**
In transformers, the self-attention mechanism computes:

\[ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V \]

Where:
- \(Q = XW_Q\), \(K = XW_K\), \(V = XW_V\) are the query, key, and value projections of the input \(X\).
- \(W_Q, W_K, W_V\) are learnable weight matrices.

#### **LoRA in Attention**
LoRA introduces low-rank updates \(\Delta W_Q\) and \(\Delta W_V\) for \(W_Q\) and \(W_V\), respectively:

\[ W_Q' = W_Q + \Delta W_Q \quad \text{and} \quad W_V' = W_V + \Delta W_V \]

Applying LoRA to both the query and value matrices ensures the model adapts effectively to the task, as these components directly control what information the model focuses on (queries) and how it retrieves knowledge (values).

---

### **6. Amplification, Not Addition**

LoRA does not add new knowledge to the model. Instead, it **amplifies existing parameters** that are relevant for the specific task. This is why LoRA is described as a "spotlight" that highlights the necessary knowledge already present in the pre-trained model.

This amplification is represented by:

\[ y = Wx + (AB^\top)x \]

Here, \((AB^\top)x\) selectively boosts specific components of the output \(Wx\).

---

### **7. Computational and Storage Efficiency**

#### **Storage Savings**
In traditional fine-tuning, the entire weight matrix \(W\) is updated and stored for each task. With LoRA, only the low-rank matrices \(A\) and \(B\) are stored, reducing storage requirements dramatically. For example:
- Full fine-tuning: 350 GB
- LoRA fine-tuning: 35 MB

#### **Computational Savings**
LoRA reduces the number of trainable parameters and gradients, leading to faster training and lower memory usage. Additionally, since \(W\) is frozen, there’s no need to compute its gradients.

---

### **8. Intuition: Library Analogy**

Think of an LLM as a vast library:
- Without LoRA: Searching for a specific book is like flipping through every page of every book (updating all parameters).
- With LoRA: LoRA provides a targeted index, directing you to the exact shelf and book needed for the task.

---

### **9. Conclusion**

LoRA leverages the low intrinsic rank of task-specific updates to fine-tune LLMs efficiently. By introducing low-rank updates to frozen weight matrices, it:
- Dramatically reduces computational and storage costs.
- Enhances task performance by amplifying existing model knowledge.
- Reveals the structured nature of how LLMs represent and adapt knowledge.

This method is particularly impactful in resource-constrained environments, enabling fine-tuning of massive models like GPT and BERT on commodity hardware.

---

### **Additional Insight: Rank of a Matrix**

The rank of a matrix is the number of linearly independent vectors it contains. This means you cannot combine any of them linearly to produce another. It also indicates how many columns are redundant because redundant columns can be obtained by linearly combining others.

For example, the weight matrix \(W\) is often rank-deficient, meaning it does not have full rank. If \(W\) has dimensions \(1000 \times 1000\), but its actual rank is 10, most of the information in \(W\) can be captured using a \(10 \times 10\) matrix.

### **In-depth Explanation of LoRA (Low-Rank Adaptation) with Mathematical Foundations**

LoRA (Low-Rank Adaptation) is an efficient fine-tuning method for large language models (LLMs). Instead of updating all the parameters of the model, LoRA introduces a small set of additional parameters that are fine-tuned to adapt the model to a specific task. This reduces computational requirements and storage while maintaining high performance. Let’s explore it step-by-step with the necessary mathematical background.

---

### **1. Key Idea Behind LoRA**

LLMs are typically parameterized by extremely large matrices, W, in each layer. These matrices define the weights that transform inputs to outputs across layers. Fine-tuning traditionally involves updating all these parameters for a specific task, which is computationally expensive and storage-heavy.

LoRA’s key observation: **The task-specific updates to these large matrices have low intrinsic rank**. This means that instead of modifying the full W, the changes can be expressed as a low-rank decomposition.

---

### **2. Mathematical Formulation of LoRA**

Let W be a weight matrix in the model. During fine-tuning, instead of directly updating W, LoRA expresses the update as:

W' = W + ΔW

Where:
- W is the pre-trained weight matrix (frozen during LoRA fine-tuning).
- ΔW is the task-specific update matrix, which LoRA approximates using a low-rank decomposition.

#### **Low-Rank Decomposition**
The update ΔW is parameterized as:

ΔW = A × Bᵀ

Where:
- A is a low-rank matrix with dimensions (output size × rank).
- B is a low-rank matrix with dimensions (input size × rank).
- The rank, r, is much smaller than the dimensions of W, making ΔW a low-rank approximation.

#### **Effective Parameter Reduction**
The number of parameters introduced by LoRA is:

Parameters in ΔW = r × (output size + input size)

This is significantly smaller than the full (output size × input size) parameters of W.

---

### **3. Training with LoRA**

During fine-tuning:
- W is frozen, meaning it is not updated.
- Only the parameters A and B are trained, which significantly reduces computational costs.

The forward pass with LoRA becomes:

y = W × x + (A × Bᵀ) × x

Where:
- x is the input vector.
- y is the output vector.

#### **Backpropagation**
Only the gradients for A and B are computed during backpropagation, while W remains unchanged. This further reduces the memory and computational overhead.

---

### **4. Intrinsic Rank and Task-Specific Adaptation**

#### **Intrinsic Rank Hypothesis**
LoRA is based on the hypothesis that task-specific updates to a pre-trained LLM are inherently low-rank. This is because:
- Pre-trained models already encode a vast amount of general knowledge.
- Fine-tuning for a specific task typically requires only minor adjustments to highlight the relevant features.

The rank, r, determines the expressiveness of the update. Empirical studies have shown that even very small values of r (e.g., r = 4 or r = 8) can lead to significant performance improvements.

---

### **5. Application in Self-Attention Mechanism**

LoRA has been particularly effective when applied to the **query** and **value** matrices in the self-attention mechanism of transformers.

#### **Attention Mechanism Overview**
In transformers, the self-attention mechanism computes:

Attention(Q, K, V) = softmax((Q × Kᵀ) / √dₖ) × V

Where:
- Q, K, V are the query, key, and value projections of the input, calculated as Q = X × Wₓ (and similarly for K and V).
- Wₓ are learnable weight matrices.

#### **LoRA in Attention**
LoRA introduces low-rank updates ΔW_Q and ΔW_V for W_Q and W_V, respectively:

W_Q' = W_Q + ΔW_Q  
W_V' = W_V + ΔW_V

Applying LoRA to both the query and value matrices ensures the model adapts effectively to the task, as these components directly control what information the model focuses on (queries) and how it retrieves knowledge (values).

---

### **6. Amplification, Not Addition**

LoRA does not add new knowledge to the model. Instead, it **amplifies existing parameters** that are relevant for the specific task. This is why LoRA is described as a "spotlight" that highlights the necessary knowledge already present in the pre-trained model.

Mathematically, this amplification is represented by:

y = W × x + (A × Bᵀ) × x

Here, (A × Bᵀ) × x selectively boosts specific components of the output W × x.

---

### **7. Computational and Storage Efficiency**

#### **Storage Savings**
In traditional fine-tuning, the entire weight matrix W is updated and stored for each task. With LoRA, only the low-rank matrices A and B are stored, reducing storage requirements dramatically. For example:
- Full fine-tuning: 350 GB
- LoRA fine-tuning: 35 MB

#### **Computational Savings**
LoRA reduces the number of trainable parameters and gradients, leading to faster training and lower memory usage. Additionally, since W is frozen, there’s no need to compute its gradients.

---

### **8. Intuition: Library Analogy**

Think of an LLM as a vast library:
- Without LoRA: Searching for a specific book is like flipping through every page of every book (updating all parameters).
- With LoRA: LoRA provides a targeted index, directing you to the exact shelf and book needed for the task.

---

### **9. Conclusion**

LoRA leverages the low intrinsic rank of task-specific updates to fine-tune LLMs efficiently. By introducing low-rank updates to frozen weight matrices, it:
- Dramatically reduces computational and storage costs.
- Enhances task performance by amplifying existing model knowledge.
- Reveals the structured nature of how LLMs represent and adapt knowledge.

This method is particularly impactful in resource-constrained environments, enabling fine-tuning of massive models like GPT and BERT on commodity hardware.

Let me break down LoRA (Low-Rank Adaptation) in a structured way that builds from fundamentals to advanced concepts.

The Core Challenge
Consider the challenge of adapting large language models. When we have a massive model like GPT-3 with hundreds of billions of parameters, we face several key problems. Each parameter is a floating-point number that needs to be adjusted during traditional fine-tuning. This means storing complete copies of the model for each adaptation, leading to enormous storage requirements – imagine needing 350GB for each version of the model. Beyond storage, the computational costs of adjusting billions of parameters are staggering, and loading multiple versions of a model becomes practically impossible.

The Mathematical Insight
This is where LoRA's elegance comes into play. Instead of modifying the entire weight matrix W (which transforms inputs in transformer models), LoRA introduces a clever mathematical decomposition. We add a small update matrix ΔW to the original weights, but here's the key insight: this update matrix can be broken down into the product of two much smaller matrices.

Mathematically, it looks like this:
W' = W + ΔW
where ΔW = BA

The matrices B and A are much smaller than W because they exploit a fundamental insight about how neural networks adapt to new tasks. B has dimensions d×r, and A has dimensions r×k, where r (the rank) is chosen to be much smaller than either d or k. Think of it like compressing a high-resolution image – while the original might have millions of pixels, you can often capture the essential features with much less information.

Understanding Low Rank
The concept of "low rank" is crucial but often misunderstood. In linear algebra, a matrix's rank represents its fundamental complexity – how many independent pieces of information it contains. Imagine trying to describe Earth's surface – while it exists in three dimensions, you only need two numbers (latitude and longitude) to specify any location. Similarly, LoRA suggests that the changes needed to adapt a language model for specific tasks have low intrinsic complexity.

This isn't just about making small changes – it's about making precise, targeted modifications that leverage the model's existing knowledge structure. Think of it like adjusting the settings on a sophisticated camera rather than rebuilding its optics.

Practical Implementation
In a transformer model, we have several types of weight matrices: query projection (Wq), key projection (Wk), value projection (Wv), output projection (Wo), and feed-forward layers (W1 and W2). Through careful experimentation, researchers discovered that applying LoRA to both query and value matrices yields optimal results. This suggests these matrices play a special role in how models adapt to new tasks.

The implementation includes several crucial details:

First, we apply a scaling factor α to control the magnitude of adaptation:
W' = W + α(BA)

Second, we initialize the matrices A and B using a normal distribution with a small standard deviation. This ensures the adaptation starts small and grows naturally during training.

Third, we typically choose ranks between 4 and 256, with 8 or 16 being common starting points. The choice of rank represents a balance – too small and it can't capture necessary adaptations, too large and we lose the benefits of parameter efficiency.

Advanced Developments
The success of LoRA has led to several important extensions. AdaLoRA dynamically adjusts the rank based on parameter importance, like a car's suspension automatically adjusting to road conditions. QLoRA combines LoRA with 4-bit quantization, further reducing memory requirements while maintaining performance.

One particularly interesting property is the ability to compose multiple adaptations:
W' = W + B₁A₁ + B₂A₂ + ...
This means we can combine task-specific adaptations, like having multiple lens filters that can be combined for different photographic effects.

Theoretical Implications
LoRA's effectiveness reveals something profound about neural networks. The fact that such small, focused changes can significantly alter model behavior suggests these models might operate more like content-addressable memories than traditional programs. It's as if we're discovering that these massive neural networks have a hidden structure that makes them more manageable than their size would suggest.

The success of targeting specific matrices (query and value) implies that knowledge in these models is organized in structured ways we're just beginning to understand. It's like discovering that a seemingly complex system actually follows simple, underlying principles.

Future Directions
Current research is exploring several exciting paths. Dynamic LoRA systems might automatically adjust their adaptation strategy based on the task at hand. Cross-model LoRA techniques could allow adaptations to transfer between different base models. Perhaps most importantly, researchers are working to develop better theoretical frameworks to explain why and when LoRA works so well.

Practical Considerations
When implementing LoRA, several factors require careful attention. The rank selection process involves balancing adaptability with efficiency. Learning rates can typically be larger than in full fine-tuning since fewer parameters are being updated. While LoRA dramatically reduces parameter count, attention mechanics still demand significant memory resources.

Understanding LoRA this deeply not only helps in its practical application but also provides insights into the fundamental nature of large language models and how they can be efficiently adapted for specific tasks. It represents a breakthrough in our ability to work with these massive models, making them more practical and accessible while maintaining their powerful capabilities.

### **In-depth Explanation of LoRA (Low-Rank Adaptation) with Mathematical Foundations**

LoRA (Low-Rank Adaptation) is an efficient fine-tuning method for large language models (LLMs). Instead of updating all the parameters of the model, LoRA introduces a small set of additional parameters that are fine-tuned to adapt the model to a specific task. This reduces computational requirements and storage while maintaining high performance. Let’s explore it step-by-step with the necessary mathematical background.

---

### **1. Key Idea Behind LoRA**

LLMs are typically parameterized by extremely large matrices, \( W \), in each layer. These matrices define the weights that transform inputs to outputs across layers. Fine-tuning traditionally involves updating all these parameters for a specific task, which is computationally expensive and storage-heavy.

LoRA’s key observation: **The task-specific updates to these large matrices have low intrinsic rank**. This means that instead of modifying the full \( W \), the changes can be expressed as a low-rank decomposition.

---

### **2. Mathematical Formulation of LoRA**

Let \( W \in \mathbb{R}^{d_{\text{out}} \times d_{\text{in}}} \) be a weight matrix in the model. During fine-tuning, instead of directly updating \( W \), LoRA expresses the update as:

\[
W' = W + \Delta W
\]

Where:
- \( W \) is the pre-trained weight matrix (frozen during LoRA fine-tuning).
- \( \Delta W \) is the task-specific update matrix, which LoRA approximates using a low-rank decomposition.

#### **Low-Rank Decomposition**
The update \( \Delta W \) is parameterized as:

\[
\Delta W = A B^\top
\]

Where:
- \( A \in \mathbb{R}^{d_{\text{out}} \times r} \) (low-rank matrix with rank \( r \)).
- \( B \in \mathbb{R}^{d_{\text{in}} \times r} \) (low-rank matrix with rank \( r \)).
- \( r \ll \min(d_{\text{out}}, d_{\text{in}}) \): The rank \( r \) is much smaller than the dimensions of \( W \), making \( \Delta W \) a low-rank approximation.

#### **Effective Parameter Reduction**
The number of parameters introduced by LoRA is:

\[
\text{Parameters in } \Delta W = r \cdot (d_{\text{out}} + d_{\text{in}})
\]

This is significantly smaller than the full \( d_{\text{out}} \cdot d_{\text{in}} \) parameters of \( W \).

---

### **3. Training with LoRA**

During fine-tuning:
- \( W \) is frozen, meaning it is not updated.
- Only the parameters \( A \) and \( B \) are trained, which significantly reduces computational costs.

The forward pass with LoRA becomes:

\[
y = W x + (A B^\top) x
\]

Where:
- \( x \in \mathbb{R}^{d_{\text{in}}} \) is the input.
- \( y \in \mathbb{R}^{d_{\text{out}}} \) is the output.

#### **Backpropagation**
Only the gradients for \( A \) and \( B \) are computed during backpropagation, while \( W \) remains unchanged. This further reduces the memory and computational overhead.

---

### **4. Intrinsic Rank and Task-Specific Adaptation**

#### **Intrinsic Rank Hypothesis**
LoRA is based on the hypothesis that task-specific updates to a pre-trained LLM are inherently low-rank. This is because:
- Pre-trained models already encode a vast amount of general knowledge.
- Fine-tuning for a specific task typically requires only minor adjustments to highlight the relevant features.

The rank \( r \) determines the expressiveness of the update. Empirical studies have shown that even very small values of \( r \) (e.g., \( r = 4 \) or \( r = 8 \)) can lead to significant performance improvements.

---

### **5. Application in Self-Attention Mechanism**

LoRA has been particularly effective when applied to the **query** and **value** matrices in the self-attention mechanism of transformers.

#### **Attention Mechanism Overview**
In transformers, the self-attention mechanism computes:

\[
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V
\]

Where:
- \( Q = XW_Q \), \( K = XW_K \), \( V = XW_V \) are the query, key, and value projections of the input \( X \).
- \( W_Q, W_K, W_V \) are learnable weight matrices.

#### **LoRA in Attention**
LoRA introduces low-rank updates \( \Delta W_Q \) and \( \Delta W_V \) for \( W_Q \) and \( W_V \), respectively:

\[
W_Q' = W_Q + \Delta W_Q \quad \text{and} \quad W_V' = W_V + \Delta W_V
\]

Applying LoRA to both the query and value matrices ensures the model adapts effectively to the task, as these components directly control what information the model focuses on (queries) and how it retrieves knowledge (values).

---

### **6. Amplification, Not Addition**

LoRA does not add new knowledge to the model. Instead, it **amplifies existing parameters** that are relevant for the specific task. This is why LoRA is described as a "spotlight" that highlights the necessary knowledge already present in the pre-trained model.

Mathematically, this amplification is represented by:

\[
y = W x + (A B^\top) x
\]

Here, \( (A B^\top) x \) selectively boosts specific components of the output \( W x \).

---

### **7. Computational and Storage Efficiency**

#### **Storage Savings**
In traditional fine-tuning, the entire weight matrix \( W \) is updated and stored for each task. With LoRA, only the low-rank matrices \( A \) and \( B \) are stored, reducing storage requirements dramatically. For example:
- Full fine-tuning: \( 350 \, \text{GB} \)
- LoRA fine-tuning: \( 35 \, \text{MB} \)

#### **Computational Savings**
LoRA reduces the number of trainable parameters and gradients, leading to faster training and lower memory usage. Additionally, since \( W \) is frozen, there’s no need to compute its gradients.

---

### **8. Intuition: Library Analogy**

Think of an LLM as a vast library:
- Without LoRA: Searching for a specific book is like flipping through every page of every book (updating all parameters).
- With LoRA: LoRA provides a targeted index, directing you to the exact shelf and book needed for the task.

---

### **9. Conclusion**

LoRA leverages the low intrinsic rank of task-specific updates to fine-tune LLMs efficiently. By introducing low-rank updates to frozen weight matrices, it:
- Dramatically reduces computational and storage costs.
- Enhances task performance by amplifying existing model knowledge.
- Reveals the structured nature of how LLMs represent and adapt knowledge.

This method is particularly impactful in resource-constrained environments, enabling fine-tuning of massive models like GPT and BERT on commodity hardware.

Let me explain LoRA (Low-Rank Adaptation) in more detail, clarifying some key concepts and correcting a few misconceptions from the text.

LoRA is a parameter-efficient fine-tuning method that was introduced to make LLM adaptation more practical and efficient. Here's a detailed breakdown:

Core Concept:
- Instead of fine-tuning all parameters in a large language model (which could be billions of parameters), LoRA introduces small trainable "rank decomposition" matrices into the model's layers.
- These matrices are much smaller than the original model weights but can effectively adapt the model's behavior for specific tasks.

The Mathematics Behind It:
- In a standard transformer model, we have weight matrices (W) that transform inputs.
- LoRA decomposes the update to these weights (ΔW) into a product of two smaller matrices: ΔW = BA, where:
  - B is a matrix of size (d × r)
  - A is a matrix of size (r × k)
  - r is the "rank" of the decomposition, typically much smaller than d and k
  - This dramatically reduces the number of parameters that need to be trained

The "Low Rank" Insight:
- The "low rank" in LoRA refers to the discovery that effective model adaptations often have low intrinsic dimensionality.
- This means that while the original model might have millions or billions of parameters, the changes needed to adapt it for a specific task can be captured in a much smaller space.
- It's not just about making small changes, but about making precise, targeted modifications that leverage the model's existing knowledge.

Implementation in Practice:
- LoRA is typically applied to specific parts of the transformer architecture:
  - The Query and Value matrices in the self-attention mechanism are the most common targets
  - Research has shown that adapting both Query and Value matrices together provides better results than adapting either alone
- The original model weights remain frozen, and only the LoRA parameters are trained

Benefits:
1. Storage Efficiency:
   - Traditional fine-tuned models require storing a complete copy of the model weights
   - LoRA only needs to store the small rank decomposition matrices
   - This can reduce storage requirements from hundreds of gigabytes to mere megabytes

2. Computational Efficiency:
   - Training is faster because fewer parameters need to be updated
   - The low-rank nature of the updates means less computational overhead during inference

3. Performance:
   - Despite its efficiency, LoRA often achieves comparable or better performance than full fine-tuning
   - The targeted nature of the updates can help prevent catastrophic forgetting

Knowledge Representation:
- LoRA's success suggests that LLMs organize knowledge in structured ways
- Rather than adding new information, LoRA often works by emphasizing or reprioritizing existing knowledge within the model
- This is more like adjusting the focus or attention of the model rather than teaching it entirely new concepts

Practical Applications:
- Task-specific adaptations (e.g., medical domain specialization)
- Language adaptation
- Personality or style tuning
- Instruction following improvements

Limitations:
- Not all tasks benefit equally from LoRA
- The optimal rank size can vary by task and model
- May not be as effective for tasks requiring substantial new knowledge

This understanding of LoRA has led to various extensions and improvements, such as QLoRA (Quantized LoRA) and AdaLoRA (Adaptive LoRA), which further optimize the approach for different scenarios and requirements.

Let me explain LoRA (Low-Rank Adaptation) comprehensively by breaking it down into stages of understanding.

First, let's understand traditional neural network training and fine-tuning:

In a typical neural network, information flows from input through multiple hidden layers to produce an output. When we train the network, we compare this output to our target (ground truth) to calculate loss, then use backpropagation to adjust all the weights throughout the network. Think of it like tuning every string on a massive piano – each weight needs individual attention and adjustment.

When we want to adapt a pre-trained model to new data, we traditionally use fine-tuning. However, this approach faces three major challenges:

The first challenge is computational cost. Modern language models can have billions of parameters, and updating all of them requires immense computational power. Imagine trying to repaint an entire skyscraper – it requires massive resources and time.

The second challenge is memory constraints. During training, we need to store not just the model weights, but also their gradients and optimizer states in GPU memory. This is like needing enough physical space to lay out and work on millions of puzzle pieces simultaneously.

The third and most subtle challenge is catastrophic forgetting. When we modify all weights, the model might forget its original learning while adapting to new tasks. It's similar to how a person might forget their native language while intensively learning a new one.

Now, let's understand how LoRA solves these problems:

LoRA introduces a brilliant insight: instead of modifying the entire weight matrix W, we can approximate the changes we want to make using two smaller matrices, A and B. Let's use concrete numbers to understand this:

Suppose we have a weight matrix W that is 1000 × 5000, containing 5 million parameters. Instead of modifying all these parameters, LoRA creates:
- Matrix B: 1000 × r
- Matrix A: r × 5000
where r is a small number like 5 or 10

The key mathematical insight is that when we multiply B and A, we get a matrix of the same size as W. So BA can be added to W to modify the model's behavior. It's like having a small adjustment panel that can control a much larger system.

Using r = 5 as an example:
- Original parameters: 1000 × 5000 = 5,000,000
- LoRA parameters: (1000 × 5) + (5 × 5000) = 30,000

This brings several powerful benefits:

1. Parameter Efficiency: We're only training 30,000 parameters instead of 5 million – a 99.4% reduction! This makes training much faster and more efficient.

2. Memory Savings: Since we're only storing and updating two small matrices, the memory requirements are drastically reduced. This allows us to use larger batch sizes or train on less powerful hardware.

3. Quick Task Switching: For different tasks (like Java vs SQL), we only need to switch out the small A and B matrices rather than loading entire new copies of W. Think of it like switching lenses on a camera rather than buying a new camera for each type of photography.

4. Original Knowledge Preservation: Since W remains frozen, we don't risk catastrophic forgetting. The model maintains its original capabilities while learning new ones.

The brilliant insight of LoRA is recognizing that many weights in large models are redundant for specific tasks. By using low-rank approximation through matrices A and B, we can capture the essential adaptations we need while dramatically reducing the computational and memory burden.

Would you like me to elaborate on any particular aspect, such as how the rank r is chosen or how LoRA handles multiple layers in practice?

I'll help explain LoRA (Low-Rank Adaptation) and why it's such an important innovation for fine-tuning large language models. Let's break this down step by step.

First, let's understand the traditional fine-tuning approach and its challenges:

In traditional neural network training and fine-tuning, we start with a model that has multiple layers of weights. When we fine-tune the model, we need to adjust all these weights through backpropagation to adapt the model to new data. This creates several significant problems:

1. The computational cost is enormous because we're updating millions or billions of parameters. Imagine trying to carefully adjust every knob in a massive control room simultaneously - it requires tremendous computing power and time.

2. Memory becomes a major constraint because we need to store not just the model weights, but also their gradients and optimizer states in GPU memory. This is like needing enough desk space to spread out and work on every page of a thousand books at once.

3. Most concerningly, the model can experience "catastrophic forgetting" - like a student who forgets their basic math while learning calculus. The model might lose its previously learned capabilities while adapting to new tasks.

This is where LoRA comes in with an elegant solution. Looking at the diagram, here's how LoRA works:

Instead of modifying the original model's weights (matrix W), LoRA introduces two smaller matrices, A and B, while keeping the original model frozen. The key insight is mathematical: if W is a d×k matrix (say 1000×5000), instead of modifying all 5 million parameters, we can create:
- Matrix B: d×r matrix (1000×r)
- Matrix A: r×k matrix (r×5000)
- Where r is much smaller than both d and k

The magic happens because:
1. When we multiply A and B, we get a matrix of the same size as W
2. We can add this result to the original W matrix's output
3. We only need to train A and B, while keeping W frozen

Let me use an analogy: Imagine W is like a massive library catalog system. Instead of reorganizing the entire catalog (traditional fine-tuning), LoRA is like adding a small appendix (matrices A and B) that efficiently describes the changes we want to make. The original catalog stays intact, but we can still effectively update the system's behavior.

The benefits are profound:
- Instead of training millions of parameters, we might only train a few thousand
- Memory usage drops dramatically
- The original model's knowledge is preserved since we never modify W
- We can store different A and B matrices for different tasks, making the model adaptable without conflicts

The equation r << min(d,k) in the diagram is crucial - it tells us that r should be much smaller than both d and k, ensuring we maintain the efficiency benefits of this approach.

This innovation has made it practical for researchers and developers to fine-tune large language models on limited hardware, democratizing access to AI technology while maintaining good performance. Would you like me to elaborate on any particular aspect of LoRA, such as how the matrices A and B are initialized or how the training process works in more detail?

In [ ]:
# ! pip install datasets
# ! pip install transformers
# ! pip install peft
# ! pip install trl

In [ ]:
import os

os.environ["WANDB_DISABLED"] = "true"  # weights and boases comes enabled by default

In [ ]:
from datasets import load_dataset
import torch
from transformers import (
    Qwen2VLForConditionalGeneration,
    Qwen2VLProcessor,
    BitsAndBytesConfig,  # quatinzation
)


from peft import (
    LoraConfig,
    get_peft_model,
)  # as we dont have enough gpu to work with full vision model so we are gpoing with adapaters such as LoRa


from trl import SFTConfig, SFTTrainer


import warnings

warnings.filterwarnings("ignore")

In [ ]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cpu"
print(f"{device=}")


MODEL_ID = "Qwen/Qwen2-VL-7B-Instruct"

EPOCHS = 1
BATCH_SIZE = 1


GRADIENT_CHECKPOINTING = True
USE_REENTRANT = False
OPTIM = "paged_adamw_32bit"
LEARNING_RATE = 2e-5
LOGGING_STEPS = 50
EVAL_STEPS = 50
SAVE_STEPS = 50
SAVE_STRATEGY = "steps"
EVAL_STRATEGY = "steps"
METRIC_FOR_BEST_MODEL = (
    "eval_loss"  # to get better model which might not be the last saved model
)
LOAD_BEST_MODEL_AT_END = True
MAX_GRAD_NORM = 1
WARMUP_STEPS = 0
DATASET_KWARGS = {
    "skip_prepare_dataset": True
}  # we have to put for VLM    # we are preparnig data oujrselvess

REMOVE_UNUSED_COLUMNS = False  # VLM thing
MAX_SEQ_LEN = 128  # max seq len of the generated text
DATA_POINTS_IN_DATASET = 283
NUM_STEPS = (DATA_POINTS_IN_DATASET // BATCH_SIZE) * EPOCHS
print(f"{NUM_STEPS=}")

## fomrat data

In [ ]:
# our data will need to have format that is compatible with VLM and SIFT trainer
# so for every data point in dataset we will the points in a fomrat that we want
# for that first we'll display a system_message

system_message = """ You are a highly advanced Vision Language Model (VLM), specializing in analyzing, describing and interpreting visual data.
Your task is to process and extract meaningful insights from images, videos and visual patterns,
leveraging multimodal understanding to provide accurate and contextually relevant infomration."""
print(f"{system_message=}")


def format_data(sample):
    # print(f"input_sample: {sample}")
    return [
        {
            "role": "system",
            "content": [
                {"type": "text", "text": system_message},
            ],
        },
        {
            "role": "user",  # input
            "content": [
                {"type": "image", "image": sample["image"]},
                {"type": "text", "text": sample["query"]},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": sample["label"][0]}],
        },
    ]

In [ ]:
train_dataset, eval_dataset, test_dataset = load_dataset(
    "HuggingFaceM4/ChartQA",
    split=["train[:1%]", "val[:1%]", "test[:1%]"],  # only using 1% of data
)
print(f"{train_dataset=} \n {test_dataset=} \n {eval_dataset=}")
print("-" * 30)
print(f"\n\n\n{train_dataset[0]=} \n\n {test_dataset[0]=} \n\n {eval_dataset[0]=}")

In [ ]:
test_dataset[0]["image"]  # reading image directly

In [ ]:
test_dataset[11]["image"]

In [ ]:
test_dataset[0]["image"]

In [ ]:
test_dataset[11]["image"]

In [ ]:
eval_dataset[0]["image"]

In [ ]:
# lets f
train_dataset[0]

In [ ]:
format_data(train_dataset[0])

In [ ]:
train_dataset = [format_data(sample=sample) for sample in train_dataset]
eval_dataset = [format_data(sample=sample) for sample in eval_dataset]
test_dataset = [format_data(sample=sample) for sample in test_dataset]
len(train_dataset), len(eval_dataset), len(test_dataset)

The code snippet you shared prepares a dataset for fine-tuning a Vision Language Model (VLM) like **Qwen/Qwen2-VL-7B-Instruct**. Let's break it down to explain why certain steps are necessary:

---

### **Why the System Message?**
The `system_message` is a way to define the role and task of the VLM. By explicitly stating that the model specializes in visual and textual analysis, it sets the context for the interactions during training or inference. This is especially important when working with large language models or VLMs that rely on multimodal input, as the system message serves as a prompt to steer the model's behavior.

---

### **Why Format the Data?**
The function `format_data(sample)` is crucial for structuring the dataset into a format compatible with the model's expected input. The VLM expects data in a structured sequence of **system**, **user**, and **assistant** roles, which mimic the conversational flow. Here's why each part is important:

1. **System Role**:
   - Defines the context or instructions for the model.
   - Ensures consistency in behavior across training and inference.

2. **User Role**:
   - Contains the input data the model must process.
   - Includes:
     - An **image**: The visual data for the VLM to interpret.
     - A **text query**: The specific question or task related to the image.

3. **Assistant Role**:
   - Contains the model's expected output (label or answer).
   - This is the target the model tries to predict during training.

---

### **Dataset Details**
You are loading the **ChartQA** dataset, which consists of questions about charts and graphs. Each data point includes:
- An `image` (chart/graph).
- A `query` (the question about the chart).
- A `label` (the correct answer).
- A `human_or_machine` flag (indicating the source of the data).

The dataset is split into train, validation, and test sets, with only 1% of the data being used for faster experimentation.

---

### **Why Use `format_data(train_dataset[0])`?**
The `format_data` function transforms each data point into the structured format the VLM expects during training or evaluation. For example, for the first training sample:

Input:
```python
{
    'image': <PIL image>,
    'query': 'Is the value of Favorable 38 in 2015?',
    'label': ['Yes'],
    'human_or_machine': 0
}
```

Transformed Output:
```python
[
    {
        "role": "system",
        "content": [{"type": "text", "text": system_message}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": <PIL image>},
            {"type": "text", "text": 'Is the value of Favorable 38 in 2015?'}
        ]
    },
    {
        "role": "assistant",
        "content": [{"type": "text", "text": 'Yes'}]
    }
]
```

This transformation is necessary because Qwen/Qwen2-VL models are trained to process data in this multimodal, role-based format. The `format_data` function ensures that the training data aligns with the model's architecture and training objectives.

---

### **Why Is This Necessary for Fine-Tuning?**
1. **Consistency**: Fine-tuning requires data in the same format as the pretraining phase to ensure compatibility.
2. **Multimodal Alignment**: The model processes both images and text, so the data must be structured to pair the visual and textual components correctly.
3. **Role-Based Interaction**: The system-user-assistant structure is integral to the conversational and task-solving nature of instruction-tuned VLMs.

---

If you're fine-tuning **Qwen/Qwen2-VL-7B-Instruct**, the `format_data` function prepares the dataset in a way that aligns with the model's multimodal and conversational architecture, ensuring that it learns effectively from the given examples.

Structuring training data for Vision Language Model (VLM) fine-tuning in a format like the example you provided is crucial for several reasons:

### 1. **Standardized Multimodal Input Format**
   - VLMs are designed to process multimodal data, i.e., text, images, and their combinations. Structuring the data explicitly defines the type and content of each modality (`'text'`, `'image'`, etc.).
   - This standardization helps the model understand how to parse and relate different data types, ensuring compatibility with its architecture and training objectives.

### 2. **Role-based Context Management**
   - Roles such as `'system'`, `'user'`, and `'assistant'` help delineate the context of the conversation or task.
   - This structure enables the model to distinguish between system instructions, user queries, and its own responses, maintaining clarity in multi-turn interactions.

### 3. **Modality Alignment**
   - Each example explicitly ties visual data (e.g., the image) with corresponding textual descriptions or queries. This alignment helps the model learn the relationship between visual and textual inputs, which is the core of vision-language understanding.

### 4. **Support for Complex Interactions**
   - By defining the interaction flow, the model can handle tasks that require sequential reasoning, such as interpreting an image, answering questions, or generating captions.
   - It also allows for chaining interactions, enabling models to retain and build on previous context.

### 5. **Facilitates Loss Computation**
   - During fine-tuning, the model learns to predict outputs based on the input context. A structured format helps clearly define inputs (e.g., `'user'` role) and expected outputs (e.g., `'assistant'` role), making it easier to compute loss and train the model effectively.

### 6. **Interoperability with Model APIs**
   - Many modern VLMs (like GPT-style VLMs or CLIP-based models) require input in specific formats to process multimodal data. Structuring data this way ensures compatibility with these APIs and models.

### 7. **Flexibility for Expansion**
   - This format is extensible. For example, additional modalities (e.g., audio, video) or metadata (e.g., timestamps, user IDs) can be added without disrupting the existing structure.
   - It supports future scenarios, like adding follow-up questions, explanations, or detailed analyses.

### 8. **Improved Training Efficiency**
   - Structured data ensures that the training pipeline can efficiently preprocess, batch, and feed data into the model. It reduces ambiguity and preprocessing errors during training.

### Example in Practice:
For the given structure:
```python
[
  {'role': 'system', 'content': [{'type': 'text', 'text': '...'}]},
  {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.Image object>}, {'type': 'text', 'text': '...'}]},
  {'role': 'assistant', 'content': [{'type': 'text', 'text': 'Yes'}]}
]
```

- The `'system'` role sets the task context.
- The `'user'` role defines multimodal input (image + text query).
- The `'assistant'` role specifies the model's expected response.

This format ensures the VLM learns to process visual data in the context of textual queries and generate coherent, task-specific outputs.

### Summary
Structuring data this way is necessary to align with the design principles of multimodal models, ensure compatibility with their architectures, and enable efficient, meaningful training for tasks requiring visual and textual understanding.

In [ ]:
print(train_dataset[0])

Dataset preprocessing done

## testing model before training

In [ ]:
sample_data = test_dataset[0]
print(f"{sample_data=}")
print("-" * 30)
sample_question = sample_data[1]["content"][1]["text"]
print(f"{sample_question=}")

print("-" * 30)
sample_answer = sample_data[2]["content"][0]["text"]
print(f"{sample_answer=}")

print("-" * 30)
sample_image = sample_data[1]["content"][0]["image"]
print(f"sample image:")
sample_image

## Loading model

In [ ]:
# loading model
if device == "cuda":
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = Qwen2VLForConditionalGeneration.from_pretrained(
        MODEL_ID,
        device_map="auto",
        quantization_config=bnb_config,
        use_cache=False,  # kv cache
    )
else:
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        MODEL_ID,
        use_cache=False,  # kv cache
    )

model = model.to(device)

# loading processor

In [ ]:
# loading processor
processor = Qwen2VLProcessor.from_pretrained(MODEL_ID)
processor.tokenizer.padding_side = "right"  # for auto regressive model, it is good rpactice ti set padding side to right
processor

## sample text generation

In [ ]:
sample_data

In [ ]:
def text_generator(sample_data):
    # first remove answer from sample whihc is at second instance .. i.e. assistan
    text_without_answer = sample_data[0:2]
    print(f"{text_without_answer=}")  # no answer included

    text = processor.apply_chat_template(
        text_without_answer,
        tokenize=False,
        add_generation_prompt=True,
    )
    print(f"\n\nPrompt: {text}")
    """
    This prints the formatted prompt after applying the chat template. The prompt is structured in a way that the model can understand, including:

    The system message.

    The user input (image placeholder + text question).

    A placeholder for the assistant's response.





    ----

    Prompt: <|im_start|>system
    You are a highly advanced Vision Language Model (VLM), specializing in analyzing, describing and interpreting visual data.
    Your task is to process and extract meaningful insights from images, videos and visual patterns,
    leveraging multimodal understanding to provide accurate and contextually relevant information.<|im_end|>
    <|im_start|>user
    <|vision_start|><|image_pad|><|vision_end|>How many food items are shown in the bar graph?<|im_end|>
    <|im_start|>assistant
    """
    print("-" * 30)

    # image input
    image_inputs = sample_data[1]["content"][0]["image"]
    print(f"{image_inputs=}")

    print("-" * 30)
    inputs = processor(
        text=[text], images=image_inputs, return_tensors="pt"  # pytorch tensor
    )

    # print(f"{inputs=}")
    print(
        """\n
        generated input:
            input_ids: Tokenized text input (as a tensor).
            attention_mask: Attention mask for the text input (as a tensor).
            pixel_values: Processed image data (as a tensor).
            image_grid_thw: Image grid dimensions (as a tensor).
        """
    )

    # putin git on device
    inputs = inputs.to(device)

    print("-" * 30)
    # now generate text
    generated_ids = model.generate(**inputs, max_new_tokens=MAX_SEQ_LEN)
    print(f"{generated_ids=}")
    # to make it sensible
    print("-" * 30)
    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
    print(f"{output_text=}")

    print("-" * 30)
    del inputs

    actual_answer = sample_data[2]["content"][0]["text"]
    return output_text[0], actual_answer


# # Excepted output: '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>\n<|im_start|>assistant\n'

generated_text, actual_answer = text_generator(sample_data)

print(f"{generated_text=}")
print(f"{actual_answer=}")

In [ ]:
print(generated_text)

In [ ]:
print(actual_answer)

# model genraeteas assistant

### generated input
input_ids: Tokenized text input (as a tensor).

attention_mask: Attention mask for the text input (as a tensor).

pixel_values: Processed image data (as a tensor).

image_grid_thw: Image grid dimensions (as a tensor).



This is a JSON-like representation of a multimodal input and task for a Vision Language Model (VLM). Let’s break it down step by step:

### 1. **System Role and Context**
   - **Role:** `system`
   - **Content:** The system's description sets the context for the VLM's capabilities:
     - Specializes in analyzing, describing, and interpreting visual data.
     - Processes images, videos, and visual patterns to extract insights.
     - Provides accurate and contextually relevant information using multimodal understanding.

### 2. **User Role and Input**
   - **Role:** `user`
   - **Content:**
     - **Image Input:** A bar graph image represented as a `PIL.PngImagePlugin.PngImageFile` object with dimensions 850x600 in RGBA mode. This indicates the image is ready for processing.
     - **Text Query:** "How many food item is shown in the bar graph?" This is the task the user wants the VLM to perform.

### 3. **Prompt Representation**
   - **`<|im_start|>` and `<|im_end|>`:** These delimiters indicate the start and end of input sections for the VLM. The sections include:
     - System instructions.
     - User inputs (both visual and textual).

### 4. **Encoded Input Details**
   - The **image** is included as an input object (`image_inputs`) for processing.
   - The **text query** is tokenized into `input_ids` for the model to process alongside the image.
   - The encoding appears to include placeholders and tokenized information used for multimodal processing.

### 5. **Task**
   - The model is tasked with interpreting the bar graph and answering the query about the number of food items displayed in it.

---

### Explanation of Specific Details:
1. **PIL Image Object:** This shows that the input image is being handled by the Python Imaging Library (PIL), indicating a preprocessing step before feeding the image to the model.
2. **Tokenized Inputs:** The `input_ids` contain the tokenized version of the text query, formatted for the model’s multimodal processing.

### Output Expectation:
The VLM will:
1. Analyze the provided bar graph image.
2. Count the number of food items depicted.
3. Return the result as a textual response to the user's query.

Let me know if you'd like to delve deeper into any specific part of this input structure!

The output represents the input and processing steps for a Vision-Language Model (VLM). Here's a concise breakdown:

1. **System Role**: The model is defined as a Vision-Language Model specializing in analyzing visual data and providing insights.
   
2. **User Input**:
   - An **image** (a bar graph, loaded using the PIL library) with metadata indicating size and mode.
   - A **text query**: *"How many food items are shown in the bar graph?"*

3. **Prompt Details**:
   - The system and user inputs are structured using specific tags (`<|im_start|>`, `<|vision_start|>`) to process multimodal inputs.
   - The query asks the model to interpret the image and provide a relevant response.

4. **Intermediate Tokens**:
   - The `input_ids` represent the tokenized form of the combined image and text input, which the model processes for generating the output.

In essence, this output outlines how the VLM processes an image and text query together, encoding them into a unified representation for interpretation and response.

# LoRA

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,  # for scaling
    lora_dropout=0.1,
    r=8,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)


# before and after putting adapters


print(f"Before adapter parameters:{ model.num_parameters()}")
# as we will be freezing original weights and just training adpters, we exexpect rtainable paramters to be a samll number

peft_model = get_peft_model(
    model, peft_config=peft_config
)  # this will convert our model into LoRA adapter model
peft_model.print_trainable_parameters()

# training args

In [ ]:
training_args = SFTConfig(
    output_dir="./output",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_checkpointing=GRADIENT_CHECKPOINTING,
    learning_rate=LEARNING_RATE,
    logging_steps=LOGGING_STEPS,
    eval_steps=EVAL_STEPS,
    eval_strategy=EVAL_STRATEGY,
    save_strategy=SAVE_STRATEGY,
    save_steps=SAVE_STEPS,
    metric_for_best_model=METRIC_FOR_BEST_MODEL,
    load_best_model_at_end=LOAD_BEST_MODEL_AT_END,
    max_grad_norm=MAX_GRAD_NORM,
    warmup_steps=WARMUP_STEPS,
    dataset_kwargs=DATASET_KWARGS,
    max_seq_length=MAX_SEQ_LEN,
    remove_unused_columns=REMOVE_UNUSED_COLUMNS,
    optim=OPTIM,
)

"""
    EPOCHS = 1
    BATCH_SIZE = 1
    GRADIENT_CHECKPOINTING = True
    USE_REENTRANT = False
    OPTIM = "paged_adamw_32bit"
    LEARNING_RATE = 2e-5
    LOGGING_STEPS = 50
    EVAL_STEPS = 50
    SAVE_STEPS = 50
    SAVE_STRATEGY = "steps"
    EVAL_STRATEGY = "steps"
    METRIC_FOR_BEST_MODEL = (
        "eval_loss"  # to get better model which might not be the last saved model
    )
    LOAD_BEST_MODEL_AT_END = True
    MAX_GRAD_NORM = 1
    WARMUP_STEPS = 0
    DATASET_KWARGS = {
        "skip_prepare_dataset": True
    }  # we have to put for VLM    # we prepared data ourselves

    REMOVE_UNUSED_COLUMNS = False  # VLM thing
    MAX_SEQ_LEN = 128  # max seq len of the generated text
    DATA_POINTS_IN_DATASET = 283
    NUM_STEPS = (DATA_POINTS_IN_DATASET // BATCH_SIZE) * EPOCHS

"""

# as we have said in DATASET_KWARGS that we will do data preparations ourselves
# we need to pass the fuction that will do data preparation into trainer.

In [ ]:
# it will return dict that will have input_ids, attention_mask, pixel values and labels
collate_sample = [train_dataset[0], train_dataset[1]]  # for batch size 2


def collate_fn(examples):
    # we will also need to pass answer
    texts = [
        processor.apply_chat_template(example, tokenize=False) for example in examples
    ]
    image_inputs = [example[1]["content"][0]["image"] for example in examples]

    batch = processor(
        text=texts,
        images=image_inputs,
        return_tensors="pt",
        padding=True,  # since it is batch
    )

    print(f"Before: {batch.keys()=}")

    labels = batch[
        "input_ids"
    ].clone()  # input_ids are token ids.. and we need ignore padding token
    labels[labels == processor.tokenizer.pad_token_id] = -100
    batch["labels"] = batch["input_ids"]
    print(f"labels added{batch.keys()=}")

    return batch


collated_data = collate_fn(collate_sample)
collated_data.keys()

This code snippet is designed to prepare a dataset for training a model, particularly a Vision-Language Model (VLM) like **Qwen2**, which processes both text and image data. The goal is to create a batch of text and image data, tokenize the text, and format it in a way suitable for model input.

### Detailed Explanation:

1. **Creating a Sample Batch (`collate_sample`)**:
   ```python
   collate_sample = [train_dataset[0], train_dataset[1]]  # for batch size 2
   ```
   - `collate_sample` is a list containing two examples (`train_dataset[0]` and `train_dataset[1]`).
   - This will be used as a batch of size 2. Each example is assumed to be a tuple containing text and image data.
   
2. **Collate Function Definition (`collate_fn`)**:
   ```python
   def collate_fn(examples):
   ```
   - `collate_fn` is the function that processes a batch of data (in this case, two samples) and prepares it for input to the model.

3. **Processing Text Data**:
   ```python
   texts = [
       processor.apply_chat_template(example, tokenize=False) for example in examples
   ]
   ```
   - `processor.apply_chat_template` is applied to each example in `examples` to prepare the text data.
   - `tokenize=False` indicates that the text should not be tokenized at this stage; tokenization will happen later (during the batch processing).

4. **Extracting Image Data**:
   ```python
   image_inputs = [example[1]["content"][0]["image"] for example in examples]
   ```
   - `image_inputs` extracts the image data from each example. It assumes the structure of each example is a tuple where the second element is a dictionary with a key `"content"`, which holds the image data.

5. **Processing Text and Image Inputs**:
   ```python
   batch = processor(
       text=texts,
       images=image_inputs,
       return_tensors="pt",
       padding=True,  # since it is batch
   )
   ```
   - `processor` is likely an instance of a tokenizer and image processor (e.g., from Hugging Face's `transformers` library). It processes both text and image inputs.
   - The `text` argument contains the list of text data (`texts`), and `images` contains the image data (`image_inputs`).
   - `return_tensors="pt"` specifies that the output should be in PyTorch tensor format.
   - `padding=True` ensures that text sequences are padded to the same length for the batch.

6. **Printing Batch Keys Before Labeling**:
   ```python
   print(f"Before: {batch.keys()=}")
   ```
   - This prints the keys of the `batch` dictionary before labels are added. These keys will likely include things like `input_ids`, `attention_mask`, and `pixel_values`.

7. **Creating Labels**:
   ```python
   labels = batch["input_ids"].clone()
   labels[labels == processor.tokenizer.pad_token_id] = -100
   batch["label"] = batch["input_ids"]
   ```
   - A clone of the `input_ids` (the tokenized text) is made to create the `labels`.
   - Any padding tokens in the `input_ids` are replaced with `-100`. This is because in most NLP models, `-100` is used to mask out padding tokens during loss computation, so they are not considered in the backpropagation.
   - The `labels` are then added to the `batch` dictionary under the key `"label"`. This ensures that the model has the correct labels to compute loss during training.

8. **Printing Batch Keys After Labeling**:
   ```python
   print(f"label added{batch.keys()=}")
   ```
   - This prints the keys of the `batch` dictionary after the `label` has been added. The batch now contains keys for the input text (`input_ids`), attention masks (`attention_mask`), image pixel values (`pixel_values`), and the `labels`.

9. **Returning the Processed Batch**:
   ```python
   return batch
   ```
   - The function returns the processed batch, which now contains:
     - `input_ids`: The tokenized text data.
     - `attention_mask`: A mask that tells the model which tokens are padding.
     - `pixel_values`: The processed image data.
     - `labels`: The labels used for training, where padding tokens are masked out with `-100`.

10. **Using the Collate Function**:
    ```python
    collated_data = collate_fn(collate_sample)
    collated_data.keys()
    ```
    - The `collate_fn` is called with the sample batch `collate_sample`.
    - The `keys()` method prints the keys of the resulting dictionary (`collated_data`), which will contain the keys for the processed text and image data (`input_ids`, `attention_mask`, `pixel_values`, `labels`).

### Summary of the Process:
- **Input**: A batch of two examples, each containing text and image data.
- **Text Processing**: The text is processed using a custom method (`apply_chat_template`), without tokenizing at this point.
- **Image Processing**: The images are extracted and prepared for the model.
- **Batch Preparation**: The text and image data are passed through a processor that tokenizes the text, processes the images, and pads the sequences.
- **Labels**: A copy of the tokenized text (`input_ids`) is used as labels, with padding tokens replaced by `-100`.
- **Output**: A batch dictionary is returned, containing the processed text (`input_ids`), image pixel values (`pixel_values`), attention masks (`attention_mask`), and labels (`label`).

### Expected Output of `collated_data.keys()`:
- `input_ids`: Tokenized IDs of the text data.
- `attention_mask`: Mask to indicate non-padding tokens.
- `pixel_values`: Processed image data (likely as tensors).
- `labels`: The same as `input_ids`, but with padding tokens masked out (`-100`).

This function is essential for preparing the dataset in a format that the model can process, ensuring the correct handling of both text and image inputs.

# Training

In [ ]:
trainer = SFTTrainer(
    model=model,  # regular mogel not LoRA
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
    processing_class=processor.tokenizer,
)

# inital evaluation of the model without any training


In [ ]:
print("-" * 30)
print("Initial evaluation")
metric = trainer.evaluate()
print(f"{metric=}")
print("-" * 30)

## training

In [ ]:
print("\nTraining")
trainer.train()
print("-" * 30)

In [ ]:
trainer.save_model(training_args.output_dir)

# clearing out memory

In [ ]:
# https://huggingface.co/learn/cookbook/en/fine_tuning_vlm_trl
import gc
import time


def clear_memory():
    # Delete variables if they exist in the current global scope
    if "inputs" in globals():
        del globals()["inputs"]
    if "model" in globals():
        del globals()["model"]
    if "processor" in globals():
        del globals()["processor"]
    if "trainer" in globals():
        del globals()["trainer"]
    if "peft_model" in globals():
        del globals()["peft_model"]
    if "bnb_config" in globals():
        del globals()["bnb_config"]
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")


clear_memory()

# Inference

In [ ]:
# load saved model


# loading model
if device == "cuda":
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = Qwen2VLForConditionalGeneration.from_pretrained(
        MODEL_ID,
        device_map="auto",
        quantization_config=bnb_config,
        use_cache=True,  # kv cache can be used in inference
    )
else:
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        MODEL_ID,
        use_cache=True,  # kv cache
    )

model = model.to(device)

model

In [ ]:
## mounitng loRa adapters onto the model
print(f"parameters before loading adapters: {model.num_parameters()}")
model.load_adapter("./output")

print(f"parameters before loading adapters: {model.num_parameters()}")

In [ ]:
# generate text
generated_text, actual_answer = text_generator(sample_data=sample_data)
print(f"generated text: {generated_text}")
print(f"actual answer: {actual_answer}")

# ***https://huggingface.co/learn/cookbook/en/fine_tuning_vlm_trl***

The **Bits and Bytes** library by Hugging Face is a lightweight utility for optimizing the memory usage and performance of large language models (LLMs). It focuses on quantization techniques (like 8-bit and 4-bit quantization) to reduce model size and accelerate inference while maintaining accuracy, making it ideal for deploying large models on resource-constrained devices.

The **Bits and Bytes** library by Hugging Face helps with fine-tuning large language models by enabling **low-bit quantization**, such as 8-bit or 4-bit, which reduces the memory footprint and computational requirements. Here's how it supports fine-tuning:

1. **Efficient Memory Usage**: Quantization reduces the size of model weights, allowing fine-tuning of larger models on GPUs with limited VRAM.

2. **Faster Training**: Lower precision computations are faster, speeding up the fine-tuning process without significantly sacrificing model performance.

3. **Parameter-Efficient Fine-Tuning (PEFT)**: It supports methods like **LoRA** (Low-Rank Adaptation) and other PEFT techniques, enabling you to fine-tune only specific parts of the model (e.g., adapters) while keeping the rest of the model frozen, further reducing resource demands.

4. **Wide Model Support**: Works seamlessly with popular Hugging Face Transformers, allowing users to quantize and fine-tune a wide range of pre-trained models.

In summary, **Bits and Bytes** makes fine-tuning more accessible by reducing hardware requirements and enabling efficient training workflows for large-scale models.